In [1]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install numpy


In [4]:
pip uninstall torch torchvision torchaudio -y
pip uninstall torch torchvision torchaudio -y


SyntaxError: invalid syntax (975789142.py, line 1)

In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 1.3/110.9 MB 41.0 MB/s eta 0:00:03
    --------------------------------------- 2.6/110.9 MB 33.7 MB/s eta 0:00:04
   - -------------------------------------- 4.0/110.9 MB 32.0 MB/s eta 0:00:04
   - -------------------------------------- 5.4/110.9 MB 31.4 MB/s eta 0:00:04
   -- ------------------------------------- 6.9/110.9 MB 31.3 MB/s eta 0:00:04
   -- ------------------------------------- 8.3/110.9 MB 31.0 MB/s eta 0:00:04
   --- ------------------------------------ 9.7/110.9 MB 30.9 MB/s eta 0:00:04
   --- ------------------------------------ 11.0/110.9 MB 29.7 MB/s eta 0:00:04
   ---- ----------------------------------- 12.3/110.9 MB 29.7 MB/s eta 0:00:04
   ---- ----------------------------------- 13.7/110.9 MB 29.8 MB/s eta 0:00:04
   ----- ---------------------------------- 15.1/110.9 MB 29.8 MB/s eta 0:00:04

In [3]:
import torch
import torchvision.transforms as T
from PIL import Image
import pandas as pd
import os, json
import shutil

# --------------------------
# CONFIG
# --------------------------
IMAGE_FOLDER = "c:/Users/thund/Downloads/dog_emotion_image_dataset/Dog Emotion/all/"          # your .jpg images
LABEL_CSV = "c:/Users/thund/Downloads/dog_emotion_image_dataset/Dog Emotion/labels.csv"          # your labels file
OUTPUT_FOLDER = "c:/Users/thund/Downloads/dog_emotion_image_dataset/Dog Emotion/cleaned_images" # where to save dog-only images
OUTPUT_CSV = "c:/Users/thund/Downloads/dog_emotion_image_dataset/Dog Emotion/cleaned_labels.csv"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --------------------------
# LOAD MODEL + LABELS
# --------------------------
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

labels = json.load(open('imagenet_class_index.json'))

# Image preprocessing
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --------------------------
# DOG DETECTION FUNCTION
# --------------------------
def is_dog(image_path, threshold=0.25):
    try:
        img = Image.open(image_path).convert('RGB')
        tensor = transform(img).unsqueeze(0)

        with torch.no_grad():
            output = model(tensor)
            probs = torch.nn.functional.softmax(output[0], dim=0)
            top5 = probs.topk(5)
    except:
        return False  # corrupted file

    # Any top-5 prediction is a dog class?
    for idx, prob in zip(top5.indices, top5.values):
        class_name = labels[str(idx.item())][1]
        if "dog" in class_name.lower() and prob.item() > threshold:
            return True

    return False

# --------------------------
# LOAD LABEL CSV
# --------------------------
df = pd.read_csv(LABEL_CSV)

# Track valid rows
valid_rows = []

# --------------------------
# PROCESS IMAGES
# --------------------------
for i, row in df.iterrows():
    filename = row["filename"]
    img_path = os.path.join(IMAGE_FOLDER, filename)

    if not os.path.exists(img_path):
        print("Missing:", filename)
        continue

    if is_dog(img_path):
        # keep the image
        shutil.copy(img_path, os.path.join(OUTPUT_FOLDER, filename))
        valid_rows.append(row)
        print("Kept:", filename)
    else:
        print("Removed:", filename)

# --------------------------
# SAVE CLEAN LABEL CSV
# --------------------------
clean_df = pd.DataFrame(valid_rows)
clean_df.to_csv(OUTPUT_CSV, index=False)

print("\nCleaning complete!")
print(f"Saved dog images to: {OUTPUT_FOLDER}")
print(f"Saved cleaned labels to: {OUTPUT_CSV}")


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Users\thund\anaconda3\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.